In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"C:\Users\alex\Documents\repos\exess\smolkaa.github.io\api\grids"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.LocalStructured3DGrid

```
[1] struct LocalStructured3DGrid{T<:Real} <: AbstractLocalStructured3DGrid
[2] LocalStructured3DGrid([T::Type,] xrange::Tuple, yrange::Tuple, zrange::Tuple, N_x::Int64, N_y::Int64, N_z::Int64)
[3] LocalStructured3DGrid([T::Type,] xrange::Tuple, yrange::Tuple, z2::Real, N_x::Int64, N_y::Int64, N_z::Int64)
```

Local structured volume grid (3D) of type `LocalStructured3DGrid{T}` over a rectangular domain. The domain is defined by tuples of the form `xrange=(x1, x2)`, `yrange=(y1, y2)`,  and `zrange=(z1, z2)`. The number of grid points in each direction is given by `N_x`,  `N_y`, and `N_z`. The grid points are centered in their respective grid element.

| Field    | Type                                | Description                           |
|:-------- |:----------------------------------- |:------------------------------------- |
| `xrange` | `Tuple{T, T}`                       | x domain limits                       |
| `yrange` | `Tuple{T, T}`                       | y domain limits                       |
| `zrange` | `Tuple{T, T}`                       | z domain limits                       |
| `N_x`    | `Int64`                             | number of grid points in x-direction  |
| `N_y`    | `Int64`                             | number of grid points in y-direction  |
| `N_z`    | `Int64`                             | number of grid points in z-direction  |
| `coords` | `Vector{LocalCartesianPosition{T}}` | coordinates                           |
| `area`   | `T`                                 | grid-element surface areas (xy-plane) |
| `volume` | `T`                                 | grid-element volume                   |

Note that the fields `area` and `volume` are scalar values, aplying to all grid elements. For a  correct, vectorized output, please use the grid-utility-methods `areas(grid)` and `volumes(grid)`.

To ensure expected behavior, the grid object should generally be created with the outer constructors [2] to [3].
